In [1]:
from dev_spark_session import DevSparkSession 
from get_stock_data import GetStockData

stockdata = GetStockData() 
spark = DevSparkSession().spark


ERROR StatusLogger Reconfiguration failed: No configuration found for '5ffd2b27' at 'null' in 'null'
ERROR StatusLogger Reconfiguration failed: No configuration found for 'Default' at 'null' in 'null'


:: loading settings :: url = jar:file:/Users/PC/Desktop/VS%20Code%20Repositories/azure-stock-market/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/PC/.ivy2/cache
The jars for the packages stored in: /Users/PC/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d5d63cc8-3c05-47c2-8fc5-175537019588;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 86ms :: artifacts dl 3ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   

In [2]:
from pyspark.sql.functions import col,explode,sha2,concat_ws,trim, lit

### Bronze -> Silver

In [3]:
## create table 
silver_path = "/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Silver/delta-table"  # Target location for Silver Delta table


spark.sql( f"""
    CREATE TABLE my_delta_table (
        Symbol STRING,
        ExchangeName STRING,
        Currency STRING,
        Type STRING,
        ExchangeTimeZone STRING,
        Volume STRING,
        High STRING,
        Low STRING,
        Close STRING,
        Open STRING,
        Date STRING,
        __CurrentFlag BOOLEAN,
        __DeletedFlag BOOLEAN,
        __EffectiveStartDateTime TIMESTAMP,
        __EffectiveEndDateTime TIMESTAMP,
        __lastmodified TIMESTAMP,
        __HashKey STRING,
        __HashValue STRING
)
USING DELTA
LOCATION  '{silver_path}';          
        """)


# ## re-create table 


# truncate_schema = spark.read.format("delta").load(silver_path).schema

# truncate_silver = spark.createDataFrame([], schema= truncate_schema)
# truncate_silver.write.format("delta").mode("overwrite").save(silver_path)



25/04/20 16:15:10 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/04/20 16:15:10 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
25/04/20 16:15:11 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
25/04/20 16:15:11 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore PC@127.0.0.1
25/04/20 16:15:11 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException


AnalysisException: The specified schema does not match the existing schema at file:/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Silver/delta-table.

== Specified ==
root
 |-- Symbol: string (nullable = true)
 |-- ExchangeName: string (nullable = true)
 |-- Currency: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- ExchangeTimeZone: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- __CurrentFlag: boolean (nullable = true)
 |-- __DeletedFlag: boolean (nullable = true)
 |-- __EffectiveStartDateTime: timestamp (nullable = true)
 |-- __EffectiveEndDateTime: timestamp (nullable = true)
 |-- __lastmodified: timestamp (nullable = true)
 |-- __HashKey: string (nullable = true)
 |-- __HashValue: string (nullable = true)


== Existing ==
root
 |-- Symbol: str

In [ ]:

# from pyspark.sql.types import StructType, StructField, DateType, StringType, DecimalType, IntegerType


# schema = StructType([StructField('Symbol', StringType(), True), 
#             StructField('ExchangeName', StringType(), True), 
#             StructField('Currency', StringType(), True), 
#             StructField('Type', StringType(), True), 
#             StructField('ExchangeTimeZone', StringType(), True), 
#             StructField('Volume', StringType(), True), 
#             StructField('High', StringType(), True), 
#             StructField('Low', StringType(), True), 
#             StructField('Close', StringType(), True), 
#             StructField('Open', StringType(), True), 
#             StructField('Date', StringType(), True)])

df_silver = spark.createDataFrame( df_bronze , schema = schema )

df_silver.show()

KeyError: 'numTargetRowsInserted'